In [1]:
import os
import json
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt

from zerobnl import CoSim

sim = CoSim()

In [2]:
data_power_grid_folder = "PowerGridData"

loads = pd.DataFrame(json.load(open(os.path.join(data_power_grid_folder, 'load.json'))))
loads.index = map(int, loads.index)
loads.head()

,name,bus,p_kw,q_kvar,const_z_percent,const_i_percent,sn_kva,scaling,in_service,type
0,LoadR1,2,10.0,1.0,0.0,0.0,None,1.0,True,None
1,LoadR11,12,10.0,1.0,0.0,0.0,None,1.0,True,None
10,LoadC14,37,10.0,1.0,0.0,0.0,None,1.0,True,None
11,LoadC17,40,10.0,1.0,0.0,0.0,None,1.0,True,None
12,LoadC18,41,10.0,1.0,0.0,0.0,None,1.0,True,None


In [3]:
set_attrs = [("load/{}/p_kw".format(load), "kw") for load in loads.name]
set_attrs += [("load/{}/q_kvar".format(load), "kvar") for load in loads.name]

sim.create_meta_model("MetaGrid", set_attrs, [])
sim.create_environment("EnvGrid", "wrapper_grid.py", "Dockerfile_grid")

files = [os.path.join(data_power_grid_folder, f) for f in os.listdir(data_power_grid_folder)]
sim.add_node("Grid", "MetaGrid", "EnvGrid", files=files)

In [4]:
sim.create_meta_model("MetaLoad", [], [("p_kw", "kw"), ("q_kvar", "kvar")])
sim.create_environment("EnvLoad", "wrapper_load.py", "Dockerfile_load")

for load in loads.name:
    sim.add_node(load, "MetaLoad", "EnvLoad", init_values={"loc": 5.0, "scale": 0.5})

In [5]:
for load in loads.name:
    sim.add_link(load, "p_kw", "Grid", "load/{}/p_kw".format(load))
    sim.add_link(load, "q_kvar", "Grid", "load/{}/q_kvar".format(load))

In [6]:
sim.create_sequence([[load for load in loads.name], ["Grid"]])
sim.set_time_unit("minutes")
sim.create_steps([60] * 6)

In [7]:
sim.run()

INFO :: Simulation finished in 0 min and 23 sec


In [8]:
sim.connect_to_results_db()
res_df = sim.get_list_of_available_results()
res_df.loc[res_df["IN/OUT"] == "X"]

,IN/OUT,Node,Attribute
10,X,Grid,ext_grid/Feeder/q_kvar
11,X,Grid,ext_grid/Feeder/p_kw
24,X,Grid,res_line


In [9]:
results = sim.get_results_by_pattern("X*Grid*")
for key, value in results.items():
    print(key)
    print(value)

INFO :: Matching results: Grid - ext_grid/Feeder/q_kvar
INFO :: Matching results: Grid - ext_grid/Feeder/p_kw
INFO :: Matching results: Grid - res_line


ModuleNotFoundError: No module named 'pandas.core.internals.managers'; 'pandas.core.internals' is not a package

In [ ]:
results.keys()

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(2, figsize=(9, 6))
ax[0].plot(results["X||Grid||ext_grid/Feeder/p_kw"], label="Feeder - Active Power", color="C0")
ax[0].set_ylabel("P [kW]")
ax[1].plot(results["X||Grid||ext_grid/Feeder/q_kvar"], label="Feeder - Reactive Power", color="C1")
ax[1].set_ylabel("Q [kVAR]")
fig.legend()

In [ ]:
plt.figure(figsize=(6, 4))
g = sim.get_graph()
# Draw the graph with a Kamada-Kawai force-directed layout.
nx.draw_kamada_kawai(g, with_labels=True, font_weight='bold')